In [8]:
from sklearn import datasets
from sklearn.model_selection import train_test_split, cross_val_score, ShuffleSplit
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from scipy import stats
from mlxtend.evaluate import mcnemar_table, mcnemar
import numpy as np

# Load the Wine dataset
wine = datasets.load_wine()
X = wine.data
y = wine.target

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)



In [9]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)

# Calculate the accuracy of the classifier on the test set
dt_accuracy = accuracy_score(y_test, dt.predict(X_test))

# Compute a confidence interval using a Binomial distribution
n = len(y_test)
z = 1.96  # for a 95% confidence interval
interval = z * np.sqrt((dt_accuracy * (1 - dt_accuracy)) / n)

# Print the confidence interval
print('Confidence interval for the accuracy: [{:.2f}, {:.2f}]'.format(dt_accuracy - interval, dt_accuracy + interval))

Confidence interval for the accuracy: [0.91, 1.01]


In [10]:


# Number of bootstrap repetitions
n_repetitions = 30

# Create a cross-validation method
cv = ShuffleSplit(n_splits=n_repetitions, test_size=0.3, random_state=42)

# Perform cross-validation
scores = cross_val_score(dt, X, y, cv=cv)

# Calculate the mean accuracy and the 95% confidence interval
mean_score = scores.mean()
confidence_interval = stats.t.interval(0.95, len(scores)-1, loc=mean_score, scale=stats.sem(scores))

print('Mean accuracy: {:.2f}'.format(mean_score))
print('95% confidence interval for the accuracy: [{:.2f}, {:.2f}]'.format(confidence_interval[0], confidence_interval[1]))

Mean accuracy: 0.92
95% confidence interval for the accuracy: [0.90, 0.93]


In [11]:
rf = RandomForestClassifier()

# Perform cross-validation for the Random Forest classifier
rf_scores = cross_val_score(rf, X, y, cv=cv)

# Calculate the mean accuracy for the Random Forest classifier
rf_mean_score = rf_scores.mean()

print('Mean accuracy for Random Forest: {:.2f}'.format(rf_mean_score))

# Perform a paired t-test between the scores of the Decision Tree and Random Forest classifiers
t_stat, p_value = stats.ttest_rel(scores, rf_scores)

print('t-statistic: {:.2f}'.format(t_stat))
print('p-value: {:.2f}'.format(p_value))

Mean accuracy for Random Forest: 0.98
t-statistic: -10.05
p-value: 0.00


In [12]:

# Fit the classifiers to the training data
dt.fit(X_train, y_train)
rf.fit(X_train, y_train)

# Get the predictions of the classifiers on the test data
dt_predictions = dt.predict(X_test)
rf_predictions = rf.predict(X_test)

# Create a contingency table
contingency_table = mcnemar_table(y_target=y_test, 
                                  y_model1=dt_predictions, 
                                  y_model2=rf_predictions)

# Perform McNemar's test
chi2, p_value = mcnemar(ary=contingency_table, corrected=True)

print('chi-squared:', chi2)
print('p-value:', p_value)

chi-squared: 1.3333333333333333
p-value: 0.24821307898992026
